In [1]:
import pandas as pd
import geopandas
import rasterio
import math, glob
import folium
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
import geopandas as gpd
import matplotlib.pyplot as plt
from folium import plugins
import datetime
import base64
from folium import IFrame
from pyproj import Proj, transform
import warnings
from osgeo import gdal
from datetime import datetime
import branca
import base64
from folium import IFrame
warnings.filterwarnings('ignore')

In [2]:
path_to_s2 = #path to data
s2_files = glob.glob(path_to_s2 + "/*")
s2_files.sort()

In [3]:
gdf = []
for k in range(len(s2_files)):
    path_to_vector = s2_files[k]+'/POI/point.shp'
    gdf.append(geopandas.read_file(path_to_vector))

In [19]:
loc = 'Oil Spill Events with coordinates map'
lon, lat = 27.2390273, 50.6923978
m = folium.Map(location=[lon, lat], zoom_start=6)

In [20]:
rgb = []
for k in range(len(s2_files)):
    rgb.append(glob.glob(s2_files[k]+'/RGB/*.jpg')[0])

In [21]:
legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    top: 35px;
    right: 5px;
    width: 250px;
    height: 80px;
    z-index:9999;
    font-size:14px;
    ">
    <p style="font-weight: bold;"><a style="color:blue;font-size:250%;margin-left:20px;">&diams;</a>&emsp;Found Event Location </p>
</div>
{% endmacro %}
'''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

In [22]:
maxwidth = 650
maxheight = 1000

In [23]:
for k in range(len(rgb)):
    inProj = Proj(gdf[k].crs)
    outProj = Proj('epsg:4326')
    x1,y1 = gdf[k].Lon[0],gdf[k].Lat[0]
    lon,lat = transform(inProj,outProj,x1,y1)
    encoded = base64.b64encode(open(rgb[k], 'rb').read()).decode()
    date = datetime.strptime(s2_files[k][-49:-41],'%Y%m%d').strftime('%d-%m-%Y')
    html = '''<h3 align="center">{}</h3><img src="data:image/JPG;base64,{}">'''.format(date,encoded)
    iframe = IFrame(html, width=520, height=520)
    popup = folium.Popup(iframe, max_width=maxwidth, max_height=maxheight)
    
    marker = folium.Marker([lon, lat], popup=popup, icon=folium.Icon(color='blue',icon='none'))
    marker.add_to(m)

In [24]:
m.get_root().add_child(legend)
title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(loc)   
m.get_root().html.add_child(folium.Element(title_html))
folium.LayerControl().add_to(m)
mini_map = plugins.MiniMap(toggle_display=True)
m.add_child(mini_map)
m